# Implementing VoxelNet Feature Encoder

Implementation details, I referenced the following:

1. https://github.com/baudm/VoxelNet-Keras/blob/master/model.py (MIT license)
2. https://github.com/steph1793/Voxelnet/blob/master/model.py (GPL license)
3. https://github.com/qianguih/voxelnet (No license)
4. The paper https://openaccess.thecvf.com/content_cvpr_2018/papers/Zhou_VoxelNet_End-to-End_Learning_CVPR_2018_paper.pdf

I am focusing on the feature encoding layer here specifically!

Also, we'll need to look at the NuScenes data descriptor (https://www.nuscenes.org/nuscenes) eventually. For now, let's assume some pointclouds.

---



In [24]:
import math
import os
import random
import sys
import struct
import warnings

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K


In [19]:
# open our sample pointcloud to work with 

with open('example_pc.bytes', 'rb') as ff:
    pc_bytes = ff.read()

pc = np.array(list(struct.iter_unpack('f', pc_bytes))).reshape(-1, 4)

print(
    f"Avg:\t{pc.mean(axis=0)}",
    f"Std:\t{pc.std(axis=0)}",
    f"Min:\t{pc.min(axis=0)}",
    f"Max:\t{pc.max(axis=0)}",
    sep='\n')

Avg:	[ 1.66082645 -0.12817482 -2.17517782  0.9391187 ]
Std:	[12.79711164 13.13313895  2.05009366  0.03580191]
Min:	[-49.17725372 -41.61127853  -3.78988695   0.81948   ]
Max:	[47.18362808 48.54064178  7.14718962  0.99192953]


# 1. Feature learning network

The feature learning network performs the following steps:

## Partitioning, grouping

Given a pointcloud, quantize the space into equally sized voxels, with range $D, H, W$ along axes $Z, Y, X$, and each voxel having size $v_D, v_H, v_W$ resulting in a 3D grid of size.

All points $p = (x, y, z)$ corresponding to a given $i, j, k$ in the grid are considered 'grouped'.

**Partitioning spec for Car Detection**

| Axis                           | $Z (D)$    | $Y (H)$     | $X (W)$    |
| ------------------------------ | ---------- | ----------- | ---------- |
| Range  $(D, H, W)$ 			 | $[-3, 1]$  | $[-40, 40]$ | $[0, 70.4]$|
| Voxel sizes $(v_D, v_H, v_W)$  | $0.4$      | $0.2$       | $0.2$      |
| Grid size  $(D', H', W')$      | $10$       | $400$       | $352$      |


**Partitioning spec for Pedestrian and Cyclist detection**

| Axis                           | $Z (D)$    | $Y (H)$     | $X (W)$    |
| ------------------------------ | ---------- | ----------- | ---------- |
| Range  $(D, H, W)$ 			 | $[-3, 1]$  | $[-20, 20]$ | $[0, 70.4]$|
| Voxel sizes $(v_D, v_H, v_W)$  | $0.4$      | $0.2$       | $0.2$      |
| Grid size  $(D', H', W')$      | $10$       | $200$       | $240$      |

For range $[a, b]$ and size $v$, the transform here is `(x - a) // v`, discarding `x` outside of the range `[a, b]` of course.

We can define this function, $$quantize(x, a, b v) = (x - a) // v$$

In [1]:
def quantize(x, a = - 3, b = 1, v = 0.4):
    '''Given a floating value x, quantize its value within
    range [a, b] with spacing v.
    
    :param x: The value to quantize.
    :type x: float
    :param a: The left-bound of the range.
    :type a: float
    :param b: The right-bound of the range.
    :type b: float
    :param v: The size of each quantize
    :type v: float
    
    :return: The quantized index of x
    :rtype: int
    
    Examples:
    >>> quantize(x = -3.0, a = - 3, b = 1, v = 0.4)
    0
    >>> quantize(x =  1.0, a = - 3, b = 1, v = 0.4)
    9
    >>> quantize(x =  0.3, a = - 3, b = 1, v = 0.4)
    8
    '''
    return int((x - a) // v)


## Sampling

After this, for each cell that has more than $T$ points ($T = 35$ for vehicle and $T = 45$ for pedestrian), randomly sample $T$ points from that cell.

Custom implementation: Rather than collect the points and *then* shuffle, we *shuffle* as we collect points. The idea is is to keep track of how many points we added to a cell, and if we've added more than `T` to that cell, randomly replace one element in that cell.

Because we randomly shuffle the points before doing this, there's no risk of bias.

1. Instantiate `G = (D', H', W', T, 4)` array of floats.
2. Instantiate `N = (D', H', W')` array of ints.
3. Randomly shuffle indices for incoming points `P`.
4. For each point `p in P` to add to the grid,
    1. Get indices `(i, j, k)` per the function `quantize` above.
    2. Let `t = N[z, y, x]`. Iterate `N[z, y, x] += 1`.
    3. If `t >= T`, do not add the point.
    4. Else, set `G[z, y, x, t, :] = (z, y, x, r)`.



In [31]:
def sample_points(
    pointcloud,
    D = [-3, 1],
    H = [-40, 40], 
    W = [0, 70.4],
    T = 35,
    vD = 0.4,
    vH = 0.2,
    vW = 0.2,
    flip_xyz = True
):
    '''Given a pointcloud (i.e. array of (n, 4) or (n, 3) points)
    and quantization parameters D, H, W, T, vD, vH, vW,
    return the (D, H, W, T, 4) array of selected points.
    
    :param pointcloud: Array of floats, size (n, 4) or (n, 3).
        Assuming (x, y, z).
    :type pointcloud: numpy.ndarray
    :param D: Range over Z axis, list of two floats
    :type D: list
    :param H: Range over Y axis, list of two floats
    :type H: list
    :param W: Range over X axis, list of two floats
    :type W: list
    :param T: Maximum number of points to sample
    :type T: int
    :param vD: Quantization size over Z axis
    :type vD: float
    :param vH: Quantization size over Y axis
    :type vH: float
    :param vW: Quantization size over X axis
    :type vW: float
    :param flip_xyz: If True, assume the input array is (x, y, z)
        rather than (z, y, x).
    :type flip_xyz: bool
    
    :return: Array of size (D, H, W, T, 4) (or 3)
        containing selection of points.
    :rtype: numpy.ndarray
    '''
    #pointcloud,
    #D = [-3, 1],H = [-40, 40],W = [0, 70.4],
    #T = 35,
    #vD = 0.4,vH = 0.2,vW = 0.2,
    #flip_xyz = True
    
    # Get the sizes of the grid
    Dsize = int((D[1] - D[0]) / vD)
    Hsize = int((H[1] - H[0]) / vH)
    Wsize = int((W[1] - W[0]) / vW)
    
    assert len(pointcloud.shape) == 2, "Pointcloud should be of shape (n, 4)"
    Psize = pointcloud.shape[-1]
    if Psize != 4:
        raise ValueError(f"Expected points in 4 dimensions, not {Psize}")
    
    # 1 and 2: Define our arrays
    G = np.zeros((Dsize, Hsize, Wsize, T, Psize))
    N = np.zeros((Dsize, Hsize, Wsize), dtype=np.int)
    
    # 3. Randomly shuffle incoming points P
    number_of_points = pointcloud.shape[0]
    random_indices = np.random.permutation(number_of_points)
    
    # 4. For each point in the pointcloud to add to the grid,
    for pc_index in random_indices:
        # 1. Get the indices `(i, j, k)`
        x, y, z, r = pointcloud[pc_index]
        # skip if any are out of bounds
        if not D[0] < z < D[1]:
            continue
        if not H[0] < y < H[1]:
            continue
        if not W[0] < x < W[1]:
            continue
        
        # note (i, j, k) corresponds to (Z, Y, X)
        i = quantize(z, a=D[0], b=D[1], v=vD)
        j = quantize(y, a=H[0], b=H[1], v=vH)
        k = quantize(x, a=W[0], b=W[1], v=vW)
        
        # 2. Get t and increase N
        t = N[i, j, k]
        N[i, j, k] += 1
        
        # 3 and 4 Skip if t >= T, add the point to G
        if t < T:
            G[i, j, k, t, :] = z, y, x, r
    
    return G, N


4. For each point `p in P` to add to the grid,
    1. Get indices `(i, j, k)` per the function `quantize` above.
    2. Let `t = N[i, j, k]`. Iterate `N[z, y, x] += 1`.
    3. If `t >= T`, choose `t = random_int(0, T)`.
    4. Set `G[z, y, x, t, :] = (z, y, x, r)`.
